# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


In [3]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [4]:
# This function looks rather simpler than the one from my video, because we're taking advantage of the latest Gradio updates

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [37]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [45]:
def do_booking(destination_city):
    print(f"Tool do_booking called for {destination_city}")
    with open("output.txt", "a") as f:  # Use "a" to append, "w" to overwrite
        print("Booking completed\n", file=f)
    return "Booking completed"

In [46]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

booking_function = {
    "name": "do_booking",
    "description": "Makes a ticket booking for the destination city. Call this whenever you need to make the booking, for example when customer asks 'book the flight to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to make booking for.",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [40]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function},
        {"type": "function", "function": booking_function}]

In [68]:
def chat(message, history, target_language):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason == "tool_calls":
        message_call = response.choices[0].message
        tool_response, city = handle_tool_call(message_call)
        messages.append(message_call)
        messages.append(tool_response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)

    final_message = response.choices[0].message.content
    translated = translate_text(final_message, target_language)

    # Update history if you're maintaining it (optional)
    history.append({"role": "user", "content": message})
    history.append({"role": "assistant", "content": final_message})

    return final_message, translated


In [47]:
# We have to write that function handle_tool_call:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    function_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')

    match function_name:
        case'do_booking':
            booking_result = do_booking(city)
            response_content = {
                "destination_city": city,
                "booking": booking_result
            }
        case'get_ticket_price':
            price = get_ticket_price(city)
            response_content = {
                "destination_city": city,
                "price": price
            }
        case _:
            response_content = "error: Unknown tool called"

    response = {
        "role": "tool",
        "content": json.dumps(response_content), 
        "tool_call_id": tool_call.id
    }

    return response, city

In [71]:
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column(scale=3):
            user_input = gr.Textbox(label="Enter your message")
            language_selector = gr.Dropdown(
                choices=["fr", "es", "de"], value="fr", label="Translate to"
            )
            send_button = gr.Button("Send")
            assistant_reply_box = gr.Textbox(label="Assistant Reply", lines=5)
        with gr.Column(scale=2):
            translation_output = gr.Textbox(label="Translation", lines=5)

    state = gr.State([])

    send_button.click(
        chat,
        inputs=[user_input, state, language_selector],
        outputs=[assistant_reply_box, translation_output]
    ).then(
        lambda: "", None, user_input  # Clear input after sending
    )

demo.launch()

* Running on local URL:  http://127.0.0.1:7876
* To create a public link, set `share=True` in `launch()`.


In [70]:
def translate_text(text, target_language):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": f"Translate the following text to {target_language}."},
            {"role": "user", "content": text}
        ]
    )
    return response.choices[0].message.content.strip()